In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Question 5

In [ ]:
import csv
import numpy as np
from scipy import stats
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder


driver_train_path = "/content/drive/MyDrive/Acad_Stuff/Course Work/Semester 5/FoML/Hackathon/train.csv"
driver_test_path = "/content/drive/MyDrive/Acad_Stuff/Course Work/Semester 5/FoML/Hackathon/test.csv"

# driver_train_path = "train.csv"
# driver_test_path = "test.csv"

In [ ]:
driver_train = pd.read_csv(driver_train_path)
print(driver_train.shape, "- original dataset")

# print(driver_train.dtypes)
print(driver_train.nunique(dropna = False))

(51490, 42) - original dataset
Report Number                      3256
Local Case Number                  1430
Agency Name                          10
ACRS Report Type                      3
Crash Date/Time                   39573
Route Type                           11
Road Name                          2227
Cross-Street Type                    11
Cross-Street Name                  4526
Off-Road Description               4018
Municipality                         21
Related Non-Motorist                 12
Collision Type                       19
Weather                              13
Surface Condition                    12
Light                                 9
Traffic Control                      12
Driver Substance Abuse               12
Non-Motorist Substance Abuse         10
Person ID                         49209
Injury Severity                       5
Circumstance                        140
Drivers License State                66
Vehicle ID                        49160
Vehicle D

### One hot encodings

In [ ]:
# print(driver_train['Light'].value_counts())

# light_one_hot = pd.get_dummies(driver_train['Light'], prefix='Light')

# light_one_hot.drop('Light_DARK -- UNKNOWN LIGHTING', axis='columns', inplace=True)
# light_one_hot.drop('Light_OTHER', axis='columns', inplace=True)
# light_one_hot.drop('Light_UNKNOWN', axis='columns', inplace=True)
# driver_train.drop('Light', axis='columns', inplace=True)


# driver_train = pd.concat([light_one_hot, driver_train], axis=1)

DAYLIGHT                    33617
DARK LIGHTS ON              11224
DARK NO LIGHTS               1716
DUSK                         1425
DAWN                         1222
DARK -- UNKNOWN LIGHTING      727
UNKNOWN                       480
OTHER                         375
Name: Light, dtype: int64


In [ ]:
for (columnName, columnData) in driver_train.iteritems():
    if driver_train[columnName].isnull().sum() >= 40000:
      driver_train.drop(columnName, axis='columns', inplace=True)
for (columnName, columnData) in driver_train.iteritems():
    if driver_train.dtypes[columnName] == 'object' and driver_train[columnName].nunique() >= 50:
      driver_train.drop(columnName, axis='columns', inplace=True)
    
      
# print(driver_train.nunique())
print(driver_train.isnull().sum())

print(driver_train.shape, "- after removing columns with many nulls")


Agency Name                           0
ACRS Report Type                      0
Route Type                         4968
Cross-Street Type                  5019
Collision Type                      299
Weather                            4029
Surface Condition                  5879
Light                               704
Traffic Control                    7774
Driver Substance Abuse             9095
Injury Severity                       0
Vehicle Damage Extent               397
Vehicle First Impact Location       222
Vehicle Second Impact Location      234
Vehicle Body Type                   785
Vehicle Movement                    198
Vehicle Continuing Dir             1129
Vehicle Going Dir                  1113
Speed Limit                           0
Driverless Vehicle                    0
Parked Vehicle                        0
Vehicle Year                          0
Equipment Problems                10666
Latitude                              0
Longitude                             0


In [ ]:
driver_train.drop("Agency Name", axis='columns', inplace=True)

imputer = SimpleImputer(fill_value=np.nan, strategy='most_frequent')

for (columnName, columnData) in driver_train.iteritems():
    if driver_train.dtypes[columnName] == 'object':
      driver_train[columnName] = imputer.fit_transform(driver_train[columnName].values.reshape(-1,1))[:,0]



# print(driver_train.dtypes)

# convert all strings to uppercase and label encoding
le = LabelEncoder()
for (columnName, columnData) in driver_train.iteritems():
  if driver_train.dtypes[columnName] == 'object':
    driver_train[columnName] = driver_train[columnName].str.upper()
    le.fit(driver_train[columnName])
    driver_train[columnName] = le.transform(driver_train[columnName])


print(driver_train.isnull().sum())
# print(driver_train.nunique(dropna = False))
# print(driver_train.dtypes)

# print(driver_train.nunique(dropna = False))



print(driver_train.shape, "- manual removal")

driver_train.to_csv('/content/drive/MyDrive/Acad_Stuff/Course Work/Semester 5/FoML/Hackathon/driver_train_m_new.csv', index=False)
# driver_train.to_csv('driver_train_m.csv', index=False)



ACRS Report Type                  0
Route Type                        0
Cross-Street Type                 0
Collision Type                    0
Weather                           0
Surface Condition                 0
Light                             0
Traffic Control                   0
Driver Substance Abuse            0
Injury Severity                   0
Vehicle Damage Extent             0
Vehicle First Impact Location     0
Vehicle Second Impact Location    0
Vehicle Body Type                 0
Vehicle Movement                  0
Vehicle Continuing Dir            0
Vehicle Going Dir                 0
Speed Limit                       0
Driverless Vehicle                0
Parked Vehicle                    0
Vehicle Year                      0
Equipment Problems                0
Latitude                          0
Longitude                         0
Fault                             0
dtype: int64
(51490, 25) - manual removal


In [ ]:
driver_test = pd.read_csv(driver_test_path, dtype='unicode')
id = driver_test.loc[:, 'Id']
print(driver_test.shape, " - original dataset")

# light_one_hot_test = pd.get_dummies(driver_test['Light'], prefix='Light')
# driver_test = pd.concat([light_one_hot_test, driver_test], axis=1)


# collecting all the column names from the train set to preserve only those columns in the test set
final_features = driver_train.columns.values
final_features = final_features[:-1]
driver_test = driver_test[final_features]
print(driver_test.shape, " - after dropping irrelevant features")

imputer = SimpleImputer(fill_value=np.nan, strategy='most_frequent')

for (columnName, columnData) in driver_test.iteritems():
    if driver_test.dtypes[columnName] == 'object':
      driver_test[columnName] = imputer.fit_transform(driver_test[columnName].values.reshape(-1,1))[:,0]



# convert all strings to uppercase and label encoding
le = LabelEncoder()
for (columnName, columnData) in driver_test.iteritems():
  if driver_test.dtypes[columnName] == 'object':
    # print(columnName)
    driver_test[columnName] = driver_test[columnName].str.upper()
    le.fit(driver_test[columnName])
    driver_test[columnName] = le.transform(driver_test[columnName])


# print(driver_test.isnull().sum())
# print(driver_test['Drivers License State'].unique())
# print(driver_test['Vehicle Body Type'].unique())

print(driver_test.shape, " - final shape after preprocessing")

driver_test.to_csv('/content/drive/MyDrive/Acad_Stuff/Course Work/Semester 5/FoML/Hackathon/driver_test_m_new.csv', index=False)
# driver_test.to_csv('driver_test_m.csv', index=False)


(77235, 42)  - original dataset
(77235, 24)  - after dropping irrelevant features
(77235, 24)  - final shape after preprocessing


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline



driver_train_data = driver_train.to_numpy()
driver_test_data = driver_test.to_numpy()

# extracting X and y for both the test and train sets
# driver status is the 13th index

# Split training/test sets
X = driver_train_data[:, :-1] # all rows, all columns except the last
y = driver_train_data[:, -1]  # all rows, only the last column

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

y_train_matrix = y_train.reshape(y_train.shape[0], 1)
y_test_matrix = y_test.reshape(y_test.shape[0], 1)

training_set = np.hstack((X_train, y_train_matrix))
test_set = np.hstack((X_test, y_test_matrix))

training_set = np.asarray(training_set, dtype=np.float32)
test_set = np.asarray(test_set, dtype=np.float32)


In [ ]:
import sklearn.datasets
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from random import randrange
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn.ensemble import GradientBoostingClassifier
import graphviz
import xgboost as xgb
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split
from scipy.stats import uniform, randint

def report_best_scores(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

clf = GradientBoostingClassifier()
params = {
    "learning_rate": uniform(0.02, 0.2), # default 0.1 
    "max_depth": randint(2, 8), # default 3
    "n_estimators": randint(50, 200), # default 100
    "subsample": uniform(0.6, 0.4)
}

search = RandomizedSearchCV(clf, param_distributions=params, random_state=0, n_iter=200, cv=3, verbose=1, n_jobs=1, return_train_score=True)

search.fit(X_train, y_train)

report_best_scores(search.cv_results_, 1)

Fitting 3 folds for each of 200 candidates, totalling 600 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 600 out of 600 | elapsed: 90.9min finished


Model with rank: 1
Mean validation score: 0.849 (std: 0.001)
Parameters: {'learning_rate': 0.07212573337852397, 'max_depth': 7, 'n_estimators': 168, 'subsample': 0.9834963692226372}



In [ ]:
# gaussian naive bayes
gnb = GaussianNB()
y_pred = gnb.fit(X_train, y_train).predict(X_test)

accuracy_gnb = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy_gnb)
print()

Accuracy: 0.705768110312682



In [ ]:
# adaboost
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC

svc = SVC(probability=True, kernel='linear')
DT = tree.DecisionTreeClassifier(max_depth=5)


clf = AdaBoostClassifier(n_estimators=100, random_state=0, learning_rate=0.09, base_estimator=DT)
clf.fit(X_train, y_train)
y_ada = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_ada)
print("Accuracy:", accuracy)
print()

Accuracy: 0.841781575710494



In [ ]:
# logistic regression
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline


clf = make_pipeline(StandardScaler(), SGDClassifier(max_iter=1000, tol=1e-4, learning_rate='constant', eta0=0.05))
# clf = SGDClassifier(max_iter=1000, tol=1e-4)

clf.fit(X_train, y_train)
lib_predictions = clf.predict(X_test)

accuracy = accuracy_score(y_test, lib_predictions)
print("Accuracy:", accuracy)
print()

Accuracy: 0.6771541399624522



In [ ]:
import xgboost as xgb

data_dmatrix = xgb.DMatrix(data=X_train,label=y_train)

# xg_cla = xgb.XGBClassifier(objective ='reg:logistic', colsample_bytree = 0.7, learning_rate = 0.1, max_depth = 8, alpha = 10, n_estimators = 200, subsample=0.7)
# xg_cla = xgb.XGBClassifier(objective ='reg:logistic', colsample_bytree = 0.3, learning_rate = 0.1, max_depth = 8, alpha = 10, n_estimators = 200, subsample=0.7)
# xg_cla = xgb.XGBClassifier(objective ='reg:logistic', colsample_bytree = 0.3, learning_rate = 0.08, max_depth = 10, alpha = 10, n_estimators = 200, subsample=0.8)

# xg_cla = xgb.XGBClassifier(objective ='reg:logistic', colsample_bytree = 0.4, learning_rate = 0.08, max_depth = 8, alpha = 10, n_estimators = 200, subsample=0.7)

xg_cla = xgb_model = xgb.XGBClassifier(objective="binary:logistic", random_state=0, colsample_bytree=0.7089850974714248, gamma=0.36862712129823866, learning_rate=0.06293533741875021, max_depth=7, n_estimators=171, subsample=0.8812869985868863)

# xg_cla = make_pipeline(StandardScaler(), xg_cla)

xg_cla.fit(X_train,y_train)
preds = xg_cla.predict(X_test)

accuracy_xgb = accuracy_score(y_test, preds)
print("Accuracy:", accuracy_xgb)
print()

Accuracy: 0.8498090244060336



In [ ]:
# random forest
K = 300
M = 5
N = 24030 # 2/3 of the training set

print("Random forest")
# clf = RandomForestClassifier(n_estimators=K, max_features=M, bootstrap=True, max_samples=N, random_state=0)
clf = RandomForestClassifier(n_estimators=400, max_features='auto', bootstrap=True, random_state=0)
# clf = make_pipeline(StandardScaler(), RF)

clf.fit(X_train, y_train)
lib_predictions = clf.predict(X_test)

accuracy = accuracy_score(y_test, lib_predictions)
print("Accuracy:", accuracy)
print()

Random forest
Accuracy: 0.8391273386418074



In [ ]:
print("Gradient Boosting")
GB = GradientBoostingClassifier(n_estimators=300, learning_rate=0.09, max_depth=8, random_state=0, subsample=0.8)
clf = make_pipeline(StandardScaler(), GB).fit(X_train, y_train)

predictions_1 = clf.predict(X_test)
accuracy_1 = accuracy_score(y_test, predictions_1)
print("Accuracy:", accuracy_1)
# prf_1 = precision_recall_fscore_support(y_test, predictions_1, labels=[1])
print()

Gradient Boosting
Accuracy: 0.8442416003107399



In [ ]:
print("500 Trees")
clf = GradientBoostingClassifier(n_estimators=1000, learning_rate = 0.1, random_state=0).fit(X_train, y_train)
predictions_2 = clf.predict(X_test)
accuracy_2 = accuracy_score(y_test, predictions_2)
print("Accuracy:", accuracy_2)
# prf_2 = precision_recall_fscore_support(y_test, predictions_2, labels=[1])
print()

500 Trees
Accuracy: 0.8406163008998511



In [ ]:
print("500 Trees")
clf = GradientBoostingClassifier(n_estimators=150, learning_rate=0.09, max_depth=7, random_state=0).fit(X_train, y_train)
predictions_2 = clf.predict(X_test)
accuracy_2 = accuracy_score(y_test, predictions_2)
print("Accuracy:", accuracy_2)
# prf_2 = precision_recall_fscore_support(y_test, predictions_2, labels=[1])
print()

500 Trees
Accuracy: 0.8496795494270732



In [ ]:
clf = GradientBoostingClassifier(n_estimators=150, learning_rate=0.09, max_depth=8, random_state=0).fit(X_train, y_train)
predictions_2 = clf.predict(X_test)
accuracy_2 = accuracy_score(y_test, predictions_2)
print("Accuracy:", accuracy_2)
# prf_2 = precision_recall_fscore_support(y_test, predictions_2, labels=[1])
print()

Accuracy: 0.8482553246585097



In [ ]:
clf = GradientBoostingClassifier(n_estimators=150, learning_rate=0.09, max_depth=7, random_state=0, subsample=0.8).fit(X_train, y_train)
predictions_3 = clf.predict(X_test)
accuracy_3 = accuracy_score(y_test, predictions_3)
print("Accuracy:", accuracy_3)
# prf_3 = precision_recall_fscore_support(y_test, predictions_3, labels=[1])
print()

Accuracy: 0.8490321745322716



In [ ]:

clf = GradientBoostingClassifier(n_estimators=1000, learning_rate=0.08, max_depth=3, random_state=0, subsample=0.8, max_features='sqrt').fit(X_train, y_train)
predictions_3 = clf.predict(X_test)
accuracy_3 = accuracy_score(y_test, predictions_3)
print("Accuracy:", accuracy_3)
# prf_3 = precision_recall_fscore_support(y_test, predictions_3, labels=[1])
print()

Accuracy: 0.8411342008156923



In [ ]:
print("300 Trees")
clf = GradientBoostingClassifier(learning_rate=0.2, n_estimators=100, max_depth=5, max_features='log2', random_state=10).fit(X_train, y_train)
predictions_3 = clf.predict(X_test)
accuracy_3 = accuracy_score(y_test, predictions_3)
print("Accuracy:", accuracy_3)
# prf_3 = precision_recall_fscore_support(y_test, predictions_3, labels=[1])
print()

300 Trees
Accuracy: 0.8344662393992361



In [ ]:
from sklearn.grid_search import GridSearchCV

Test set

In [ ]:
import xgboost as xgb



# clf = tree.DecisionTreeClassifier()
# clf = clf.fit(X, y)

# clf = GradientBoostingClassifier(n_estimators=300, learning_rate=0.1, max_depth=5, random_state=0).fit(X, y)

K = 1000
M = 5
N = 34327 # 2/3 of the training set
# # clf = RandomForestClassifier(n_estimators=K, max_features=M, bootstrap=True, max_samples=N, max_depth=5).fit(X, y)
# # clf = GradientBoostingClassifier(n_estimators=90, learning_rate=0.1, max_depth=8, random_state=10, loss='exponential').fit(X, y)
# # clf = GradientBoostingClassifier(learning_rate=0.1, n_estimators=60, max_depth=9, max_features='sqrt', subsample=0.8, random_state=10).fit(X, y)
# # clf = GradientBoostingClassifier(n_estimators=500, learning_rate=0.08, max_depth=8, random_state=0, loss='exponential').fit(X, y)
# # clf = GradientBoostingClassifier(n_estimators=500, learning_rate=0.05, max_depth=8, random_state=0, subsample=0.8).fit(X, y)
# clf = GradientBoostingClassifier(n_estimators=150, learning_rate=0.09, max_depth=7, random_state=0, subsample=0.8).fit(X, y)
clf = GradientBoostingClassifier(n_estimators=168, learning_rate=0.07212573337852397, max_depth=7, random_state=0, subsample=0.9834963692226372).fit(X, y)
# 'learning_rate': 0.07212573337852397, 'max_depth': 7, 'n_estimators': 168, 'subsample': 0.9834963692226372}

predictions = clf.predict(driver_test_data)

# data_dmatrix = xgb.DMatrix(data=X,label=y)
# # xg_cla = xgb.XGBClassifier(objective ='reg:logistic', colsample_bytree = 0.7, learning_rate = 0.1, max_depth = 8, alpha = 10, n_estimators = 500, subsample=0.7)
# # xg_cla = xgb.XGBClassifier(objective ='reg:logistic', colsample_bytree = 0.7, learning_rate = 0.1, max_depth = 8, alpha = 10, n_estimators = 200, subsample=0.7)
# # xg_cla = xgb.XGBClassifier(objective ='reg:logistic', colsample_bytree = 0.3, learning_rate = 0.1, max_depth = 8, alpha = 10, n_estimators = 200, subsample=0.7)
# xg_cla = xgb.XGBClassifier(objective ='reg:logistic', colsample_bytree = 0.4, learning_rate = 0.08, max_depth = 8, alpha = 10, n_estimators = 200, subsample=0.7)

# xg_cla.fit(X,y)
# predictions = xg_cla.predict(driver_test_data)

# prf = precision_recall_fscore_support(y_test, predictions, labels=[1])
final_submission = np.c_[id, predictions]
final_submission = np.r_[[['Id', 'Fault']], final_submission]
print(final_submission.shape)
print(final_submission)

(77236, 2)
[['Id' 'Fault']
 ['0.0' '1.0']
 ['1.0' '0.0']
 ...
 ['77232.0' '0.0']
 ['77233.0' '1.0']
 ['77234.0' '1.0']]


In [ ]:
predictions = predictions.astype(int)
id = id.astype(int)

final_submission = np.c_[id, predictions]
final_submission = np.r_[[['Id', 'Fault']], final_submission]
print(final_submission.shape)
print(final_submission)


# fname = '/content/drive/MyDrive/Acad_Stuff/Course Work/Semester 5/FoML/Hackathon/final_submission.csv'
fname = 'final_submission.csv'

# np.savetxt(fname, final_submission, delimiter=' ')

with open(fname, "w") as f:
    writer = csv.writer(f)
    writer.writerows(final_submission)

(77236, 2)
[['Id' 'Fault']
 ['0' '1']
 ['1' '0']
 ...
 ['77232' '0']
 ['77233' '1']
 ['77234' '1']]


In [ ]:
id = id.astype(int)
temp = np.zeros(id.shape)

temp = temp.astype(int)
id = id.astype(int)


zeros = np.c_[id, temp]
zeros = np.r_[[['Id', 'Fault']], zeros]
print(zeros.shape)
print(zeros)


fname = 'zeros.csv'
# np.savetxt(fname, zeros, delimiter=' ')

with open(fname, "w") as f:
    writer = csv.writer(f)
    writer.writerows(zeros)

(77236, 2)
[['Id' 'Fault']
 ['0' '0']
 ['1' '0']
 ...
 ['77232' '0']
 ['77233' '0']
 ['77234' '0']]
